In [1]:
import pandas as pd
import numpy as np
from scipy import stats


In [2]:
df = pd.read_csv('World University Rankings 2023.csv')

In [41]:
for i in df.select_dtypes('object').columns:
    print(i)
    if len(df[i].mode())>1:
        try:
            df[i] = pd.to_numeric(df[i])
        except ValueError as e:
            rand_index=np.random.choice(df[i][~df[i].isna()].index,1)
            #x=str(e)
            x_str=re.sub(r'\d+', '', df[i][rand_index].iloc[0])
            if (bool(re.search(r'\d+', df[i][rand_index].iloc[0])) == True) and (len(x_str)<(len(df[i][rand_index].iloc[0])-len(x_str))):
                df[i] = pd.to_numeric(df[i].str.replace(',',''),errors='coerce')
                print(f'column "{i}" has been converted to Numeric')
                continue
            else:
                continue
    else:
        continue

University Rank
Name of University
Location
International Student
Female:Male Ratio
OverAll Score


In [42]:
# len(re.sub(r'\d+', '',df['No of student'][rand_index].iloc[0]))
# x_str=re.sub(r'\d+', '', df['No of student'][rand_index].iloc[0])
# if (bool(re.search(r'\d+', df['No of student'][rand_index].iloc[0])) == True) and (len(x_str)<(len(df['No of student'][rand_index].iloc[0])-len(x_str))):
#     print('yes')\
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341 entries, 0 to 2340
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   University Rank              2341 non-null   object 
 1   Name of University           2233 non-null   object 
 2   Location                     2047 non-null   object 
 3   No of student                2209 non-null   float64
 4   No of student per staff      2208 non-null   float64
 5   International Student        2209 non-null   object 
 6   Female:Male Ratio            2128 non-null   object 
 7   OverAll Score                1799 non-null   object 
 8   Teaching Score               1799 non-null   float64
 9   Research Score               1799 non-null   float64
 10  Citations Score              1799 non-null   float64
 11  Industry Income Score        1799 non-null   float64
 12  International Outlook Score  1799 non-null   float64
dtypes: float64(7), obj

In [339]:
class Analyze_csv():
    def __init__(self,df):
        self.df = df

    def no_of_rows(self):
        return len(self.df)
        
    def no_of_null_total(self):
        x=''
        for i,j in zip(self.df.columns,self.df.isna().sum()):
            x = x+f'"{i}" column has {j} null values'+'\n'
        return x
    def col_details(self):
        total_cols=len(self.df.columns)
        num_cols='The numerical columns in this dataset are: \n'
        cat_cols='The Categorical columns in this dataset are: \n'

        for i in df.select_dtypes('number').columns:
            num_cols = num_cols+i+'\n'
        for j in df.select_dtypes('object').columns:
            cat_cols = cat_cols+j+'\n'
        return total_cols, num_cols, cat_cols

    def num_col_details(self):
        a='Mean(average) value for the Numerical columns are as below:\n'
        b='Min value for the Numerical columns are as below:\n'
        c='Max value for the Numerical columns are as below:\n'
        d='Median value for the numerical columns are as below:\n'
        e='standard deviation of the numerical columns as below:\n'
        
        for i in df.select_dtypes('number').columns:
            a=a+f'{i} : {str(round(self.df[i].mean(),2))}'+'\n'
            b=b+f'{i} : {str(round(self.df[i].min(),2))}'+'\n'
            c=c+f'{i} : {str(round(self.df[i].max(),2))}'+'\n'
            d=d+f'{i} : {str(round(self.df[i].median(),2))}'+'\n'
            e=e+f'{i} : {str(round(self.df[i].std(),2))}'+'\n'
            
        return a,b,c,d,e
    def cat_col_details(self):
        a='Mode for the Categorical columns are as below:\n'
        b='Top 5 repeated values for Categorical columns are as below:\n'
        
        for i in self.df.select_dtypes('object').columns:
            if len(self.df[i].mode()) > 1:
                a=a+f'{i} : this column has more than 1 mode'+'\n'
            else:
                a=a+f'{i} : {str(self.df[i].mode()[0])}'+'\n'
        for i in self.df.select_dtypes('object').columns:
            if len(self.df[i].mode()) == 1:
                b=b+f'{i} : {dict(self.df[i].value_counts()[0:5])}'+'\n'
            else:
                b=b+f'{i} : This column has more than 1 mode'+'\n'
                 
        return a,b
        
    def outliers(self):
        Q1 = self.df.select_dtypes('number').quantile(0.25)
        Q3 = self.df.select_dtypes('number').quantile(0.75)
        IQR = Q3 - Q1
        x='Outliers for each Numerical column are as below:\n'

        for i in df.select_dtypes('number').columns:

            x=x+f'{i} : {str(len(df[~((self.df[i]>(Q1[i]-1.5*IQR[i])) & (self.df[i]<(Q3[i]+1.5*IQR[i])))][i])-self.df[i].isna().sum())}'+'\n'
            
        return x
    
    def distribution_type(self):
        a='Skewness for each Numerical column are as below:\n'
        b='Kurtosis for each Numerical column are as below:\n'
        c='Shapiro-wilk test for each Numerical column are as below:\n'
        for i in self.df.select_dtypes('number').columns:
            a=a+f'{i} : {self.df[~self.df[i].isna()][i].skew()}\n'
            b=b+f'{i} : {self.df[~self.df[i].isna()][i].kurtosis()}\n'
            
            shapiro_test = stats.shapiro(self.df[~self.df[i].isna()][i])
            if shapiro_test[1] > 0.05:
                c = c+f'{i} : This column appear to be normally distributed\n'
            else:
                c = c+f'{i} : This column does not appear to be normally distributed\n'

        return a,b,c

    def correlation(self):
        a = 'The correlation coefficient between the numerical columns are given below:\n'
        for i in self.df.select_dtypes('number').columns:
            for j in self.df.select_dtypes('number').columns:
                corr=self.df[[i,j]].corr().iloc[0,1]
                if i!=j and (corr>0.75 or corr<-0.75):
                    a=a+f'{i} <-> {j} : {corr}\n'
                else:
                    continue
        return a     

In [340]:
a1=Analyze_csv(df)

In [351]:
prompt = f'''
[INST]
Below are the details of a dataset. it covers details like null value count, 
total rows, data type of the columns,type of distribution, type of relation 
among the columns. your task is to answer the queries on the dataset by 
referring the below details, answers the questions accurately. If user asks
about the range of certain column, refer to maximum and minimum values of the 
column and answer it.each detail is seperated by an empty line.
--------------------------------------------------------------
Details:
Total number of records in the dataset are {a1.no_of_rows()}

Null values for each column are given below\n{a1.no_of_null_total()}

Total columns in the dataset are : {a1.col_details()[0]}

{a1.col_details()[1]}

{a1.col_details()[2]}

{a1.num_col_details()[0]}

{a1.num_col_details()[1]}

{a1.num_col_details()[2]}

{a1.num_col_details()[3]}

{a1.num_col_details()[4]}

{a1.cat_col_details()[0]}

{a1.cat_col_details()[1]}

{a1.outliers()}

{a1.distribution_type()[0]}

{a1.distribution_type()[1]}

{a1.distribution_type()[2]}

{a1.correlation()}

--------------------------------------------------------------
Question: {query}
[/INST]
'''

In [352]:
print(prompt)


[INST]
Below are the details of a dataset. it covers details like null value count, 
total rows, data type of the columns,type of distribution, type of relation 
among the columns. your task is to answer the queries on the dataset by 
referring the below details, answers the questions accurately. If user asks
about the range of certain column, refer to maximum and minimum values of the 
column and answer it.each detail is seperated by an empty line.

Details:
Total number of records in the dataset are 2341

Null values for each column are given below
"University Rank" column has 0 null values
"Name of University" column has 108 null values
"Location" column has 294 null values
"No of student" column has 132 null values
"No of student per staff" column has 133 null values
"International Student" column has 132 null values
"Female:Male Ratio" column has 213 null values
"OverAll Score" column has 542 null values
"Teaching Score" column has 542 null values
"Research Score" column has 542 

In [147]:
# df['Location'].value_counts().head().index
df['Location'].mode()[0]


'United States'

In [255]:
Q1 = df.select_dtypes('number').quantile(0.25)
Q3 = df.select_dtypes('number').quantile(0.75)
IQR = Q3 - Q1

In [262]:
len(df[~((df['No of student']>(Q1['No of student']-1.5*IQR['No of student'])) & (df['No of student']<(Q3['No of student']+1.5*IQR['No of student'])))]['No of student'])-df['No of student'].isna().sum()

82

In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341 entries, 0 to 2340
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   University Rank              2341 non-null   object 
 1   Name of University           2233 non-null   object 
 2   Location                     2047 non-null   object 
 3   No of student                2209 non-null   object 
 4   No of student per staff      2208 non-null   float64
 5   International Student        2209 non-null   object 
 6   Female:Male Ratio            2128 non-null   object 
 7   OverAll Score                1799 non-null   object 
 8   Teaching Score               1799 non-null   float64
 9   Research Score               1799 non-null   float64
 10  Citations Score              1799 non-null   float64
 11  Industry Income Score        1799 non-null   float64
 12  International Outlook Score  1799 non-null   float64
dtypes: float64(6), obj

In [172]:
import re

def contains_number(string):
    # Search for any digit in the string
    return bool(re.search(r'\d', string))

# Example usage
test_string = "Hello, this is a test string with number 123,456."
print(contains_number(test_string))  # Output: True

test_string = "No numbers here!"
print(contains_number(test_string))  # Output: False


True
False


In [269]:
dict(df['Location'].value_counts()[0:4])

{'United States': 173, 'Japan': 150, 'United Kingdom': 149, 'India': 91}

In [182]:
np.random.randint(0,len(df))

859

In [225]:
bool(re.search(r'^(?=.*\d)(?=.*,)[^a-zA-Z]*$',df['No of student'][np.random.choice(df['No of student'][~df['No of student'].isna()].index,1)]))


TypeError: expected string or bytes-like object, got 'Series'

In [230]:
df['No of student'][np.random.choice(df['No of student'][~df['No of student'].isna()].index,1)].iloc[0]

'10,540'

In [289]:


# Assuming 'data' is a one-dimensional numpy array or a pandas series
shapiro_test = stats.shapiro(df[~df['No of student'].isna()]['No of student'])

print(f"Test statistic: {shapiro_test[0]}, p-value: {shapiro_test[1]}")


Test statistic: 0.5144734382629395, p-value: 0.0


In [300]:
xx=df.select_dtypes('number').corr()

In [313]:
for i in range(0,len(xx.columns)):
    xx.iloc[i,i]=0
    

In [331]:
xx[(xx>0.75) | (xx<-0.75)].idxmax(skipna=True)

C:\Users\naren\AppData\Local\Temp\ipykernel_28928\1421972568.py:1: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  xx[(xx>0.75) | (xx<-0.75)].idxmax(skipna=True)


No of student                             NaN
No of student per staff                   NaN
Teaching Score                 Research Score
Research Score                 Teaching Score
Citations Score                           NaN
Industry Income Score                     NaN
International Outlook Score               NaN
dtype: object

In [338]:
df[df.select_dtypes('number').columns].corr()

,No of student,No of student per staff,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score
No of student,1.000000,0.315527,-0.001826,0.022474,0.026195,-0.016474,-0.034050
No of student per staff,0.315527,1.000000,-0.130569,0.006686,-0.005684,-0.002588,0.029470
Teaching Score,-0.001826,-0.130569,1.000000,0.882181,0.451169,0.492579,0.385006
Research Score,0.022474,0.006686,0.882181,1.000000,0.514013,0.583051,0.537993
Citations Score,0.026195,-0.005684,0.451169,0.514013,1.000000,0.182949,0.536379
Industry Income Score,-0.016474,-0.002588,0.492579,0.583051,0.182949,1.000000,0.218060
International Outlook Score,-0.034050,0.029470,0.385006,0.537993,0.536379,0.218060,1.000000


In [354]:
response=[('id', 'chatcmpl-9Zdk8Ocj0MwO73oufFUO6b4ceBFAc'), ('choices', [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The columns in the dataset are as follows:\n\n1. University Rank\n2. Name of University\n3. Location\n4. No of student\n5. No of student per staff\n6. International Student\n7. Female Ratio\n8. OverAll Score\n9. Teaching Score\n10. Research Score\n11. Citations Score\n12. Industry Income Score\n13. International Outlook Score\n\nThese columns contain information about the universities, including various numerical and categorical data points.', role='assistant', function_call=None, tool_calls=None))]), ('created', 1718281280), ('model', 'gpt-3.5-turbo-0125'), ('object', 'chat.completion'), ('system_fingerprint', None), ('usage', CompletionUsage(completion_tokens=100, prompt_tokens=2725, total_tokens=2825))]

NameError: name 'Choice' is not defined